In [1]:
import duckdb
import os

# Construct the path to your database relative to the notebook location
DB_PATH = os.path.join("..", "data", "simulation.db")

# Connect to the DuckDB database
con = duckdb.connect(DB_PATH)

In [3]:
import pandas as pd
df = con.execute("SELECT * FROM order_events").fetchdf()
print(df)

   order_id  arrival_time  start_pick_time  end_pick_time picker_id
0         0           2.0              2.0            7.0  Picker-1
1         1           4.0              4.0            9.0  Picker-2
2         2           6.0              6.0           11.0  Picker-3
3         3           8.0              8.0           13.0  Picker-1
4         4          10.0             10.0           15.0  Picker-2
5         5          12.0             12.0           17.0  Picker-3
6         6          14.0             14.0           19.0  Picker-1
7         7          16.0             16.0           21.0  Picker-2
8         8          18.0             18.0           23.0  Picker-3
9         9          20.0             20.0           25.0  Picker-1
10       10          22.0             22.0           27.0  Picker-2
11       11          24.0             24.0           29.0  Picker-3
12       12          26.0             26.0           31.0  Picker-1
13       13          28.0             28.0      

In [17]:
cycle_time_query = """
SELECT
    AVG(end_pick_time - arrival_time)
FROM order_events
WHERE end_pick_time IS NOT NULL 
"""

workload_query = """
SELECT
    picker_id,
    COUNT(order_id)
FROM order_events
GROUP BY picker_id
"""

In [18]:
print(con.execute(cycle_time_query).fetchdf())

   avg((end_pick_time - arrival_time))
0                                  5.0


In [20]:
print(con.execute(workload_query).fetchdf())
con.close()

  picker_id  count(order_id)
0  Picker-3               19
1  Picker-2               20
2  Picker-1               20
